# This is the second step in running corridor analysis for the Caribbean portion of the blueprint



Created by Amy Keister, last run by Amy Keister on 9/6/2023. It took less than a minute to run. 

In [70]:
import os
import arcpy
import numpy

In [71]:
import time
start = time.time()

In [72]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\H_VIPRhubs.gdb"

In [96]:
# define final outputs
Hub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ha_VIPR_LM_Input.gdb\Hubs"

In [74]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [75]:
# define additional inputs
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"
PADFee= r"F:\GIS_DATA\ProtectedLands\PADUS3_0Geodatabase\PAD_US3_0.gdb\PADUS3_0Fee"
PADEasement = r"F:\GIS_DATA\ProtectedLands\PADUS3_0Geodatabase\PAD_US3_0.gdb\PADUS3_0Easement"

# Start Analysis

In [76]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [77]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\H_VIPRhubs.gdb


# Identify areas in the top 10% of the zonation runs

In [78]:
# take zonation times 100
# I only needed this on the workshop run where I didn't start with the combined zonation raster
#out_raster = arcpy.sa.Times(Zonation, 100); out_raster.save("Zonation100")

In [79]:
# convert to integer
# I only needed this on the workshop run where I didn't start with the combined zonation raster
#out_raster = arcpy.sa.Int("Zonation100"); out_raster.save("Zonation100Int")

In [81]:
# use a conditional statement to pull out the top 10%, use environemnts to clip to inland
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent, mask=Rextent):
    out_raster = arcpy.sa.Con(Zonation, 1, 0, "VALUE >90"); out_raster.save("Top10")

In [82]:
# reclassify to turn NoData into 0
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("Top10", "Value", "0 0;1 1;NODATA 0", "DATA"); out_raster.save("Top10z")

# Bring in Protected Areas
In the South Atlantic we used TNC's secured areas, and we had it pretty figured out what to keep and remove. I'm learning on 
PADUS 2.1 Combined Fee, Easement, but we need to keep an eye out for improvements
I checked and it doesn't seem like this layer includes department of defense lands like Army or Air force bases, so no need to remove them

In [83]:
# Bring in PADUS 3.0. national geodatabase download doesn't have it's own combined Fee and Easement layer, so we have to create it. Combined Fee, Easement 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Union([PADFee,PADEasement], "PadFeeEase", "ALL", None, "GAPS")

In [84]:
# add field to use to convert to raster
arcpy.management.CalculateField("PadFeeEase", "raster", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'PadFeeEase'>

In [85]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent=Rextent):
    arcpy.conversion.PolygonToRaster("PadFeeEase", "raster", "PAD", "CELL_CENTER", "NONE", Rextent)

# Combine 2 potential hub layers 

In [86]:
# combine rasters, keeping max value
out_raster = arcpy.sa.CellStatistics(["Top10z", "PAD"],\
"MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("PotHub")

# Calculate size of potential hubs

In [87]:
# use region group to create clumps of isolated potential hubs
out_raster = arcpy.sa.RegionGroup("PotHub", "EIGHT", '', '', '0'); out_raster.save("PotHubRG")

In [88]:
# pull out clumps that are larger than 2,000 ha (5,000 acres), keep their region group value
# this is for the 30 meter resolution version
#out_raster = arcpy.sa.Con("PotHub3a", "PotHub3a", '', "Count > 22482 And Link = 1"); out_raster.save("PotHub4")

In [89]:
# pull out clumps that are larger than 2,000 ha (5,000 acres), keep their region group value
# this is for the 90 meter resolution version
#out_raster = arcpy.sa.Con("PotHuba", "PotHuba", '', "Count > 2498 And Link = 1"); out_raster.save("PotHub4")

In [90]:
# pull out clumps that are larger than 200 ha (500 acres), keep their region group value
# this is for the 30 meter resolution version
out_raster = arcpy.sa.Con("PotHubRG", "PotHubRG", '', "Count > 2222 And Link = 1"); out_raster.save("PotHub200hR")

In [91]:
# convert to vector
arcpy.conversion.RasterToPolygon("PotHub200hR", "PotHub200h", "NO_SIMPLIFY", "Value", "MULTIPLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\H_VIPRhubs.gdb\\PotHub200h'>

In [92]:
#Add field that ESRI is willing to read
arcpy.management.CalculateField("PotHub200h", "UID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'PotHub200h'>

In [97]:
# copy features
arcpy.management.CopyFeatures("PotHub200h", Hub, '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ha_VIPR_LM_Input.gdb\\Hubs'>

In [ ]:
end = time.time()
print(end - start)